<h1>Dinosaur challenge</h1>

<h2>Installs</h2>

In [40]:
!pip install matplotlib
!pip install opencv-python opencv-contrib-python
!pip install numpy
!pip install tensorflow
!pip install pandas


[notice] A new release of pip available: 22.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


<h2>Imports</h2>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from collections import defaultdict, Counter
import pandas as pd

# importing the Keras libraries and packages
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom

<h2>Global variables</h2>
To quickly test different settings and the impact on the result, different variables can be set here. These will impact how the model runs.

In [98]:
source_dir = "train/train"

<h2>Analyzing data</h2>

In [99]:
class_counts = defaultdict(int)
dimension_counts = defaultdict(Counter)

# Scan all class folders
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)
    if os.path.isdir(class_path):
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)

            # Read image using OpenCV
            img = cv2.imread(image_path)

            if img is not None:
                # Update counters
                class_counts[class_name] += 1

                # Get image dimensions (width, height)
                height, width = img.shape[:2]
                dimension_counts[class_name][(width, height)] += 1
            else:
                print(f"Warning: Could not read {image_path}")

# Display results
print("\n📊 Dataset Analysis Results:")
for class_name in sorted(class_counts.keys()):
    print(f"\n🦖 Class: {class_name}")
    print(f"  - Total images: {class_counts[class_name]}")
    print(f"  - Unique dimensions:")
    for dims, count in dimension_counts[class_name].items():
        print(f"    {dims[0]}x{dims[1]}: {count} images")


📊 Dataset Analysis Results:

🦖 Class: Ankylosaurus
  - Total images: 152
  - Unique dimensions:
    474x333: 1 images
    474x280: 1 images
    474x355: 16 images
    474x474: 25 images
    474x365: 1 images
    474x281: 2 images
    474x398: 1 images
    474x219: 2 images
    474x289: 2 images
    474x251: 1 images
    474x316: 6 images
    474x296: 5 images
    474x266: 21 images
    474x225: 1 images
    400x269: 1 images
    474x269: 1 images
    474x342: 1 images
    474x424: 1 images
    474x315: 2 images
    474x536: 1 images
    474x262: 1 images
    474x236: 1 images
    474x220: 1 images
    474x468: 1 images
    474x457: 1 images
    474x480: 1 images
    474x677: 1 images
    474x265: 1 images
    474x270: 1 images
    474x264: 1 images
    474x310: 1 images
    474x710: 1 images
    474x459: 1 images
    474x356: 1 images
    474x475: 1 images
    474x295: 1 images
    474x460: 1 images
    474x194: 1 images
    474x292: 1 images
    474x271: 1 images
    474x372: 1 image

<h2>Get data</h2>

In [105]:
from tensorflow.keras.utils import image_dataset_from_directory

# Set the parameters for your data
batch_size = 32
image_size = (224, 224)
validation_split = 0.2

# Create the training dataset from the 'train/train' directory
train_ds = image_dataset_from_directory(
    directory=r'C:\Users\iljas\Desktop\ai\project2\Dino\train\train',
    labels='inferred',
    label_mode='categorical',
    batch_size=batch_size,
    image_size=image_size,
    validation_split=validation_split,
    subset='training',
    seed=123
)

# Create the validation dataset from the 'train/train' directory
validation_ds = image_dataset_from_directory(
    directory=r'C:\Users\iljas\Desktop\ai\project2\Dino\train\train',
    labels='inferred',
    label_mode='categorical',
    batch_size=batch_size,
    image_size=image_size,
    validation_split=validation_split,
    subset='validation',
    seed=123
)

# If you have a test set in the same structure:
test_ds = image_dataset_from_directory(
    directory=r'C:\Users\iljas\Desktop\ai\project2\Dino\test\test',
    labels=None,
    label_mode=None,
    batch_size=batch_size,
    image_size=image_size
)

# Ensure datasets can keep generating batches endlessly if needed
train_ds = train_ds.repeat()
validation_ds = validation_ds.repeat()
test_ds = test_ds.repeat()

Found 1269 files belonging to 7 classes.
Using 1016 files for training.
Found 1269 files belonging to 7 classes.
Using 253 files for validation.
Found 303 files.


In [107]:
NUM_CLASSES = 7
# Set the parameters for your data
batch_size = 32
image_size = (224, 224)
validation_split = 0.2
dataset_dir = "train/train"
train_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=validation_split,
    subset="training",
    seed=123,
    image_size=image_size,  # will automatically resize to 128x128
    batch_size=batch_size
)

# Load validation dataset with 20% of data
validation_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=validation_split,
    subset="validation",
    seed=123,
    image_size=image_size,
    batch_size=batch_size
)

def one_hot_encode_dataset(dataset, num_classes):
    def one_hot_map(images, labels):
        return images, tf.one_hot(labels, depth=num_classes)
    return dataset.map(one_hot_map)

train_ds = one_hot_encode_dataset(train_ds, NUM_CLASSES)
validation_ds = one_hot_encode_dataset(validation_ds, NUM_CLASSES)

# View class names
#print("Classes:", train_ds.class_names)

Found 1269 files belonging to 7 classes.
Using 1016 files for training.
Found 1269 files belonging to 7 classes.
Using 253 files for validation.


<h2>Initializing + compiling model</h2>

In [109]:
IMG_SIZE = 224

#model = tf.keras.Sequential([
#  layers.InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, 3)),
#  layers.Rescaling(1./255),
#  layers.RandomFlip("horizontal"),
#  layers.RandomZoom(0.2),
#
#  layers.Conv2D(32, (3, 3), activation="relu"),
#  layers.MaxPooling2D((2, 2)),
#  layers.Dropout(0.2),
#  layers.Conv2D(32, (3, 3), activation="relu"),
#  layers.MaxPooling2D((2, 2)),
#  layers.Dropout(0.2),
#
#  layers.GlobalAveragePooling2D(),
#  layers.Dense(128, activation="relu"),
#  layers.Dense(NUM_CLASSES, activation="softmax")
#])
model = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1),
    
    # Block 1
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2, 2),
    BatchNormalization(),

    # Block 2
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    BatchNormalization(),

    # Block 3
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    BatchNormalization(),

    # Block 4 (extra depth)
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(2, 2),
    BatchNormalization(),

    # Global pooling replaces flatten (much safer for high res images)
    GlobalAveragePooling2D(),

    # Dense classifier
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

NameError: name 'Input' is not defined

<h2>Training the network</h2>

In [103]:
model.fit(train_ds,
            validation_data = validation_ds,
            #steps_per_epoch = 100,
            #validation_steps = 20, # we have a .20 validation split, so we'll use 80% less batches for validation
            epochs = 20
          )

Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 264s 8s/step - accuracy: 0.1853 - loss: 2.1962 - val_accuracy: 0.1225 - val_loss: 2.5904
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 134s 4s/step - accuracy: 0.2021 - loss: 2.0266 - val_accuracy: 0.2016 - val_loss: 2.0338
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 118s 4s/step - accuracy: 0.2305 - loss: 1.9398 - val_accuracy: 0.1818 - val_loss: 2.0108
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 131s 4s/step - accuracy: 0.2291 - loss: 1.9179 - val_accuracy: 0.1700 - val_loss: 1.9595
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 117s 4s/step - accuracy: 0.2684 - loss: 1.8621 - val_accuracy: 0.2174 - val_loss: 1.9122
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 116s 4s/step - accuracy: 0.2196 - loss: 1.8847 - val_accuracy: 0.1858 - val_loss: 1.9599
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 118s 4s/step - accuracy: 0.2697 - loss: 1.8511 - val_accuracy: 0.2292 - val_loss: 2.0372
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 124s 4s/step - accuracy: 0.2737 - loss: 1.8491 - val_accuracy: 0.1897 - v

KeyboardInterrupt: 

<h2>Model opslaan</h2>

In [52]:
import datetime
date_str = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
filename = f'content/model{date_str}.keras'
print(filename)
model.save(filename)

content/model20250515180425.keras


In [ ]:
model = load_model("content/name")

<h2>Predictions</h2>

In [83]:
#import os
#import cv2
#import pandas as pd
#import tensorflow as tf
#import datetime

# --- Parameters ---
TEST_DIR = r'C:\Users\iljas\Desktop\ai\project2\Dino\test\test'
IMG_SIZE = (474, 474)  # Should match your model's expected input
OUTPUT_CSV = f"predictions_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

# --- Storage for predictions ---
results = []

# --- Loop through all images ---
for image_name in os.listdir(TEST_DIR):
    image_path = os.path.join(TEST_DIR, image_name)

    # Check if it's an image file
    if not image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    # Read and preprocess image using OpenCV
    img = cv2.imread(image_path)
    if img is None:
        print(f"⚠ Warning: Could not read {image_path}")
        continue

    # Resize and normalize like your model expects
    img_resized = cv2.resize(img, IMG_SIZE)
    img_normalized = img_resized / 255.0
    img_expanded = tf.expand_dims(img_normalized, axis=0)  # Add batch dimension

    # Predict
    #prediction = model.predict(img_expanded, verbose=0)
    prediction = model.predict(img_expanded, verbose=0)
    print(prediction)
    print(image_path)

    # Since your model now outputs softmax (NUM_CLASSES = 7), use argmax
    predicted_label = int(np.argmax(prediction[0]))

    # Store result (keeping image id as filename without extension)
    image_id = os.path.splitext(image_name)[0]
    results.append({'id': image_id, 'label': predicted_label})

# --- Save to CSV ---
df = pd.DataFrame(results)
df.to_csv(OUTPUT_CSV, sep=';', index=False)

[[0.12466289 0.15574275 0.14380443 0.12080763 0.12935312 0.1592706
  0.16635859]]
C:\Users\iljas\Desktop\ai\project2\Dino\test\test\1.jpg
[[0.12466289 0.15574275 0.14380443 0.12080763 0.12935312 0.1592706
  0.16635859]]
C:\Users\iljas\Desktop\ai\project2\Dino\test\test\10.jpg
[[0.12466289 0.15574275 0.14380443 0.12080763 0.12935312 0.1592706
  0.16635859]]
C:\Users\iljas\Desktop\ai\project2\Dino\test\test\100.jpg
[[0.12466289 0.15574275 0.14380443 0.12080763 0.12935312 0.1592706
  0.16635859]]
C:\Users\iljas\Desktop\ai\project2\Dino\test\test\101.jpg
[[0.12466289 0.15574275 0.14380443 0.12080763 0.12935312 0.1592706
  0.16635859]]
C:\Users\iljas\Desktop\ai\project2\Dino\test\test\102.jpg
[[0.12466289 0.15574275 0.14380443 0.12080763 0.12935312 0.1592706
  0.16635859]]
C:\Users\iljas\Desktop\ai\project2\Dino\test\test\103.jpg
[[0.12466289 0.15574275 0.14380443 0.12080763 0.12935312 0.1592706
  0.16635859]]
C:\Users\iljas\Desktop\ai\project2\Dino\test\test\104.jpg
[[0.12466289 0.15574275